In [1]:
import gradio as gr
import numpy as np
import requests
import pandas as pd
import hopsworks
import joblib
import torch
from torch import nn


import os
from dotenv import load_dotenv
import httpx
import datetime
import json
from urllib.request import Request, urlopen
import random
from datetime import datetime
import missingno as msno
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder



/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194714
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
#Load api keys
load_dotenv()
weather_api_key = os.getenv("weather_api_key")
pressure_api_key = os.getenv("pressure_api_key")
flight_api_key = os.getenv("flight_api_key")

In [4]:
#Fit label encoding to previous mapping
# Previous mapping
previous_mapping = {'ATL': 0, 'CLT': 1, 'DEN': 2, 'DTW': 3, 'EWR': 4, 'FLL': 5, 'IAD': 6, 'IAH': 7, 'JFK': 8,
                    'LAS': 9, 'LAX': 10, 'MCO': 11, 'MIA': 12, 'ORD': 13, 'PHL': 14, 'SEA': 15, 'SFO': 16}

# Create a label encoder
label_encoder = LabelEncoder()

# Fit the label encoder with the previous mappings
label_encoder.fit(list(previous_mapping.keys()))
airport_encoding_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(airport_encoding_map)


{'ATL': 0, 'CLT': 1, 'DEN': 2, 'DTW': 3, 'EWR': 4, 'FLL': 5, 'IAD': 6, 'IAH': 7, 'JFK': 8, 'LAS': 9, 'LAX': 10, 'MCO': 11, 'MIA': 12, 'ORD': 13, 'PHL': 14, 'SEA': 15, 'SFO': 16}


In [30]:
#Mappings
icao_to_iata_map = {
    "KDTW": "DTW",
    "KLAS": "LAS",
    "KPHL": "PHL",
    "KDEN": "DEN",
    "KCLT": "CLT",
    "KSEA": "SEA",
    "KMCO": "MCO",
    "KFLL": "FLL",
    "KIAD": "IAD",
    "KIAH": "IAH",
    "KSFO": "SFO",
    "KEWR": "EWR",
    "KMIA": "MIA",
    "KJFK": "JFK",
    "KLAX": "LAX",
    "KORD": "ORD",
    "KATL": "ATL",
}
iata_to_icao_map = {v: k for k, v in icao_to_iata_map.items()}
wac_map = {
    "BOS": 13,
    "CLT": 36,
    "DEN": 82,
    "DTW": 43,
    "EWR": 21,
    "FLL": 33,
    "IAD": 38,
    "IAH": 74,
    "JFK": 22,
    "LAS": 85,
    "LAX": 91,
    "MCO": 33,
    "MIA": 33,
    "ORD": 41,
    "PHL": 23,
    "SEA": 93,
    "SFO": 91,
    "ATL": 34,
}
weather_features = [
    ("dewpoint", "value"),
    "relative_humidity",
    ("remarks_info", "precip_hourly", "value"),
    ("remarks_info", "temperature_decimal", "value"),
    ("visibility", "value"),
    ("wind_direction", "value"),
    ("wind_gust", "value"),
    ("wind_speed", "value"),
]
pressure_features = [("pressure", "hpa")]
flight_features = [
    "flight_date",
    ("departure", "iata"),
    ("departure", "delay"),
    ("departure", "scheduled"),
    ("arrival", "iata"),
    ("arrival", "delay"),
    ("arrival", "scheduled"),
]
airport_id_map={
    "CLT": 11057,
    "DEN": 11292,
    "DTW": 11433,
    "EWR": 11618,
    "FLL": 11697,
    "IAD": 12264,
    "IAH": 12266,
    "JFK": 12478,
    "LAS": 12889,
    "LAX": 12892,
    "MCO": 13204,
    "MIA": 13303,
    "ORD": 13930,
    "PHL": 14100,
    "SEA": 14747,
    "SFO": 14771,
    "ATL": 10397,
}
label_tranformed_airport_id_map={'ATL': 0, 'CLT': 1, 'DEN': 2, 'DTW': 3, 'EWR': 4, 'FLL': 5, 'IAD': 6, 'IAH': 7, 'JFK': 8,
                                  'LAS': 9, 'LAX': 10, 'MCO': 11, 'MIA': 12, 'ORD': 13, 'PHL': 14, 'SEA': 15, 'SFO': 16}
# Create predefined lists for origin and destination airport codes
airports = [ "PHL - PHILADELPHIA INTERNATIONAL AIRPORT, PA US",
    "SEA - SEATTLE TACOMA AIRPORT, WA US",
    "JFK - JFK INTERNATIONAL AIRPORT, NY US",
    "DEN - DENVER INTERNATIONAL AIRPORT, CO US",
    "EWR - NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",
    "LAS - MCCARRAN INTERNATIONAL AIRPORT, NV US",
    "MCO - ORLANDO INTERNATIONAL AIRPORT, FL US",
    "ATL - ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US",
    "FLL - FORT LAUDERDALE INTERNATIONAL AIRPORT, FL US",
    "DTW - DETROIT METRO AIRPORT, MI US",
    "IAD - WASHINGTON DULLES INTERNATIONAL AIRPORT, VA US",
    "ORD - CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",
    "LAX - LOS ANGELES INTERNATIONAL AIRPORT, CA US",
    "CLT - CHARLOTTE DOUGLAS AIRPORT, NC US",
    "MIA - MIAMI INTERNATIONAL AIRPORT, FL US",
    "IAH - HOUSTON INTERCONTINENTAL AIRPORT, TX US",
    "SFO - SAN FRANCISCO INTERNATIONAL AIRPORT, CA US"]

In [6]:

#Class definition needed due to the way pytorch neural networks are saved and loaded by python
# A solution, if needed, would be to save the state dict of the NN and load the model via load_state_dict
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

#Load model from model registry
mr = project.get_model_registry()
model = mr.get_model("flight_delay_model", version=2)
model_dir = model.download()
model = joblib.load(model_dir + "/flight_delay_model.pkl")

# get the original train test splits used for training the model and use it for fitting scaler
feature_view = fs.get_feature_view(name="flight_data_v3",version=1)
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(training_dataset_version=3)

#fit scaler the same way it was used for training
scaler = StandardScaler()
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)


Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (16.67s) 


In [7]:
def get_weather_data(selected_airports_iata):
    # Input: list of selected airports in IATA code
    # Make API call to fetch weather data for the airport
    # Process and return weather data
    responses = {}
    for airport in selected_airports_iata:
        print(f"Getting weather for {airport}")
        request = Request(
            f"https://avwx.rest/api/metar/{iata_to_icao_map[airport]}",
            headers={"Authorization": weather_api_key},
        )
        response_body = urlopen(request).read()
        response_json = json.loads(response_body)
        responses[airport] = response_json

    weather_data = []

    for airport in selected_airports_iata:
        response_json = responses[airport]
        data = {"airport": airport}
        data["HourlyDewPointTemperature"] = response_json["remarks_info"][
            "dewpoint_decimal"
        ]["value"]
        data["HourlyRelativeHumidity"] = response_json["relative_humidity"]
        if response_json["remarks_info"]["precip_hourly"] is not None:
            data["HourlyPrecipitation"] = response_json["remarks_info"]["precip_hourly"][
                "value"
            ]
        else:
            data["HourlyPrecipitation"] = 0
        data["HourlyDryBulbTemperature"] = response_json["remarks_info"][
            "temperature_decimal"
        ]["value"]
        data["HourlyVisibility"] = response_json["visibility"]["value"]
        data["HourlyWindDirection"] = response_json["wind_direction"]["value"]
        if response_json["wind_gust"] is not None:
            data["HourlyWindGustSpeed"] = response_json["wind_gust"]["value"]
        else:
            data["HourlyWindGustSpeed"] = 0
        data["HourlyWindSpeed"] = response_json["wind_speed"]["value"]
        weather_data.append(data)

    weather_data = pd.DataFrame(weather_data)
    #weather_data.info()
    return weather_data

In [8]:
def get_pressure_data(selected_airports_iata):
    # Input: list of selected airports in IATA code
    responses={}
    url = "https://api.checkwx.com/metar/KJFK/decoded"

    #response = requests.request("GET", url, headers={"X-API-Key": pressure_api_key})
    for airport in selected_airports_iata:
        print(f"Getting pressure for {airport}")
        request = Request(
            f"https://api.checkwx.com/metar/{iata_to_icao_map[airport]}/decoded",
            headers={"X-API-Key": pressure_api_key},
        )
        response_body = urlopen(request).read()
        response_json = json.loads(response_body)
        responses[airport] = response_json

    pressure_data = []

    for airport in selected_airports_iata:
        response_json = responses[airport]
        data = {"airport": airport}
        data["HourlyStationPressure"] = response_json["data"][0]["barometer"]["hpa"]
        pressure_data.append(data)

    pressure_data = pd.DataFrame(pressure_data)
    #pressure_data.info()
    return pressure_data

In [9]:
def get_flight_data(origin, destination,scheduled_dep_time, scheduled_arr_time):
    # Input: origin airport IATA code, destination airport IATA code, 
    # and dep and arr time in HH:MM 24 hour format
    current_datetime = datetime.now()

    # Extract different date-related information
    day_of_week = current_datetime.weekday()  
    day_of_month = current_datetime.day  
    year = current_datetime.year  
    month = current_datetime.month



    origin_wac = wac_map[origin]
    origin_airport_id = label_tranformed_airport_id_map[origin]

    # Mapping destination to dest_WAC and dest_airport_id
    dest_wac = wac_map[destination]
    dest_airport_id = label_tranformed_airport_id_map[destination]
    # Create a DataFrame for the given airport codes
    airport_df = pd.DataFrame({
        #"Year":[year],
        "month":[month],
        "Day_of_month":[day_of_month],
        "Day_of_week":[day_of_week],
        "origin": [origin],
        "origin_airport_id": [origin_airport_id],
        "origin_WAC": [origin_wac],
        "dest": [destination],
        "dest_airport_id": [dest_airport_id],
        "dest_WAC": [dest_wac],
        "CRS_DEP_TIME":[int(scheduled_dep_time.replace(":", ""))],
        "CRS_ARR_TIME":[int(scheduled_arr_time.replace(":", ""))],
        "airport":[origin]
        
    })

    #print(airport_df.info())
    #print(airport_df)
    return airport_df

In [10]:
# Define the function to predict flight delay based on user inputs
def predict_delay(origin, destination,scheduled_dep_time, scheduled_arr_time):
    
    #test code to try running Gradio app
    origin=origin.split()[0]
    destination=destination.split()[0]
    
    #error handling
    try:
        # check if correct hour format by trying to convert to datetime objects
        datetime.strptime(scheduled_dep_time, "%H:%M")
        datetime.strptime(scheduled_arr_time, "%H:%M")
    except ValueError:
        # else error
        return "Error: Please enter scheduled departure and arrival times in 24-hour format (HH:MM)."
    if origin == destination:
        return "Error: Origin and destination airports cannot be the same. Please select different airports."
    
    #Get data from APIs
    selected_airports_iata = [origin,destination]
    weather_data=get_weather_data(selected_airports_iata)
    pressure_data=get_pressure_data(selected_airports_iata)
    flight_data=get_flight_data(origin, destination,scheduled_dep_time, scheduled_arr_time)

    #Merge data
    weather_delay_data = pd.merge(pressure_data, weather_data, on="airport")

    # fix order of columns so that it is same as in training
    weather_delay_data=weather_delay_data.reindex(sorted(weather_delay_data.columns), axis=1)
    
    #merge columns
    flight_weather_data=pd.merge(flight_data, weather_delay_data, on="airport")
    
    #drop objects
    flight_weather_data.drop(columns=['airport', 'origin', 'dest'], inplace=True)

    #fix type
    columns_to_float64 = ['HourlyPrecipitation', 'HourlyVisibility', 'HourlyWindGustSpeed', 'HourlyWindSpeed']
    for column in columns_to_float64:
        # Convert to int64
        flight_weather_data[column] = flight_weather_data[column].astype('float64')

    #flight_weather_data.info()

    flight_weather_data=torch.tensor(flight_weather_data.values, dtype=torch.float32)
    print(flight_weather_data)
    #flight_weather_data=scaler.transform(flight_weather_data.reshape(1, -1))
    flight_weather_data=scaler.transform(flight_weather_data)

    print(flight_weather_data)
    # transform np array to torch tensor
    flight_weather_data_tensor=torch.tensor(flight_weather_data, dtype=torch.float32)
    print(flight_weather_data_tensor)

    output=model(flight_weather_data_tensor)
    #return output
    return "Predicted delay for {} to {} with the scheduled departure time {} and scheduled " \
        "arrival time {} is {} minutes".format(origin, destination, scheduled_dep_time, scheduled_arr_time,int(output.item()))


In [10]:
#Test input
origin=airports[-3]
destination=airports[4]
scheduled_dep_time="17:00"
scheduled_arr_time="20:09"

In [11]:
predict_delay(origin, destination,scheduled_dep_time, scheduled_arr_time)

Getting weather for MIA


Getting weather for EWR
Getting pressure for MIA
Getting pressure for EWR
tensor([[2.0240e+03, 1.0000e+00, 3.0000e+00, 2.0000e+00, 1.3303e+04, 3.3000e+01,
         1.1618e+04, 2.1000e+01, 1.7000e+03, 2.0090e+03, 8.9000e+00, 1.3900e+01,
         0.0000e+00, 7.1811e-01, 1.0200e+03, 1.0000e+01, 3.6000e+02, 0.0000e+00,
         3.0000e+00]])
[[ 4.95464606e+00 -1.70151018e+00 -1.46191874e+00 -9.97097503e-01
   5.69504082e-01 -7.63184324e-01 -7.33785098e-01 -1.21524480e+00
   6.91570454e-01  9.30281479e-01 -2.48880890e-03 -4.13821385e-01
  -9.97782929e-02 -2.76083443e+00  6.62719390e-01 -1.78330094e+00
   1.69297292e+00 -3.55746920e-01 -4.48970006e-01]]
tensor([[ 4.9546e+00, -1.7015e+00, -1.4619e+00, -9.9710e-01,  5.6950e-01,
         -7.6318e-01, -7.3379e-01, -1.2152e+00,  6.9157e-01,  9.3028e-01,
         -2.4888e-03, -4.1382e-01, -9.9778e-02, -2.7608e+00,  6.6272e-01,
         -1.7833e+00,  1.6930e+00, -3.5575e-01, -4.4897e-01]])


'Predicted delay for MIA to EWR with the scheduled departure time 17:00 and scheduled arrival time 20:09 is 39 minutes'

In [ ]:
# Create Gradio interface with dropdowns for airport selection
iface = gr.Interface(
    fn=predict_delay,
    inputs=[
        gr.inputs.Dropdown(choices=airports, label="Origin Airport"),
        gr.inputs.Dropdown(choices=airports, label="Destination Airport"),
        gr.inputs.Textbox(type="text", label="Enter scheduled Departure time in 24-hour format HH:MM(eg. 17:59)"),
        gr.inputs.Textbox(type="text", label="Enter scheduled Arrival time in 24-hour format HH:MM (eg. 20:59)"),
    ],
    outputs=gr.outputs.Textbox(label="Predicted Departure Delay")
)

# Launch the Gradio app
iface.launch()

## Testing inference

In [31]:
def flight_api(selected_airports_iata):
    endpoint = "http://api.aviationstack.com/v1/flights"
    flight_data = []


    for i in range(len(selected_airports_iata)):
        print(f"Getting flights for {selected_airports_iata[i]}")
        # Parameters for the API request
        rand_int = random.randint(0, len(selected_airports_iata) - 1)
        params = {
            "access_key": flight_api_key,
            "dep_iata": selected_airports_iata[
                i
            ],  # Replace 'SFO' with your desired airport code
            "arr_iata": selected_airports_iata[rand_int],
            "flight_status": ["active"]
        }

        # Making the GET request
        response = requests.get(endpoint, params=params)
        # Checking if the request was successful (status code 200)
        if response.status_code == 200:
            response = response.json()  # Parsing the JSON response
            # Handle and process 'data' (flight information)
            data = {"airport": selected_airports_iata[i]}
            if len(response["data"]) == 0:
                continue
            
            rand_int = random.randint(0, len(response["data"]) - 1)

            response = response["data"][rand_int]

            
            data["DEPARTURE_DATETIME"] = response["departure"]["scheduled"]
            data["DEPARTURE_DATETIME"] = pd.to_datetime(
                data["DEPARTURE_DATETIME"]
            )
            current_datetime = datetime.now()

            # Extract different date-related information
            day_of_week = current_datetime.weekday()  
            day_of_month = current_datetime.day  
            year = current_datetime.year  
            month = current_datetime.month
            # add year, quarter, month, day_of_month, day_of_week
            #data["YEAR"] = year
            #weather_delay_data["QUARTER"] = weather_delay_data["FL_DATE"].dt.quarter
            data["MONTH"] = month
            data["DAY_OF_MONTH"] = day_of_month
            data["DAY_OF_WEEK"] = day_of_week


            data["ORIGIN"] = response["departure"]["iata"]
            # Mapping IATA codes to airport IDs for 'dest' and 'origin' columns
            
            data['origin_airport_id'] = label_tranformed_airport_id_map[data['ORIGIN']]
            data["ORIGIN_WAC"] = wac_map[data["ORIGIN"]]
            # add wac code to weather_delay_data
            data["DEST"] = response["arrival"]["iata"]


            data['dest_airport_id'] = label_tranformed_airport_id_map[data['DEST']]
            data["DEST_WAC"] = wac_map[data["DEST"]]
            



            
            
            
            
            

            # Transform date columns to datetime
            data["CRS_DEP_TIME"] = response["departure"]["scheduled"]
            data["CRS_ARR_TIME"] = response["arrival"]["scheduled"]
            
            data["airport"]=data["ORIGIN"]
            data["origin"]=data["ORIGIN"]
            data["dest"]=data["DEST"]
            
            if response["departure"]["delay"] is not None:
                data["DEP_DELAY_NEW"] = response["departure"]["delay"]
            else:
                data["DEP_DELAY_NEW"] = 0.0
            

            flight_data.append(data)
        else:
            print(response.json())
            print("Request failed:", response.status_code)

    flight_data = pd.DataFrame(flight_data)
    
    # Apply label encoding to DEST and ORIGIN columns
    #flight_data['dest_airport_id'] = label_encoder.transform(flight_data['DEST'])
    #flight_data['origin_airport_id'] = label_encoder.transform(flight_data['ORIGIN'])

    flight_data.drop(columns=[ 'ORIGIN', 'DEST','DEPARTURE_DATETIME'], inplace=True)
    flight_data.info()
    return flight_data

In [32]:
selected_airports_iata = [
    "DTW",
    "LAS",
    "PHL",
    "DEN",
    "CLT",
    "SEA",
    "MCO",
    "FLL",
    "IAD",
    "IAH",
    "SFO",
    "EWR",
    "MIA",
    "JFK",
    "LAX",
    "ORD",
    "ATL",
]

In [33]:
flight_data_api=flight_api(selected_airports_iata)
weather_data=get_weather_data(selected_airports_iata)
pressure_data=get_pressure_data(selected_airports_iata)

Getting flights for DTW
Getting flights for LAS
Getting flights for PHL
Getting flights for DEN
Getting flights for CLT
Getting flights for SEA
Getting flights for MCO
Getting flights for FLL
Getting flights for IAD
Getting flights for IAH
Getting flights for SFO
Getting flights for EWR
Getting flights for MIA
Getting flights for JFK
Getting flights for LAX
Getting flights for ORD
Getting flights for ATL
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airport            11 non-null     object
 1   MONTH              11 non-null     int64 
 2   DAY_OF_MONTH       11 non-null     int64 
 3   DAY_OF_WEEK        11 non-null     int64 
 4   origin_airport_id  11 non-null     int64 
 5   ORIGIN_WAC         11 non-null     int64 
 6   dest_airport_id    11 non-null     int64 
 7   DEST_WAC           11 non-null     int64 
 8   CRS_DEP_TIME  

In [34]:
flight_data_api.head()

,airport,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,origin,dest,DEP_DELAY_NEW
0,DEN,1,4,3,2,82,12,33,2024-01-04T10:33:00+00:00,2024-01-04T16:28:00+00:00,DEN,MIA,19
1,SEA,1,4,3,15,93,11,33,2024-01-04T08:10:00+00:00,2024-01-04T16:57:00+00:00,SEA,MCO,32
2,MCO,1,4,3,11,33,8,22,2024-01-04T13:30:00+00:00,2024-01-04T16:10:00+00:00,MCO,JFK,14
3,FLL,1,4,3,5,33,2,82,2024-01-04T15:15:00+00:00,2024-01-04T17:51:00+00:00,FLL,DEN,14
4,IAH,1,4,3,7,74,10,91,2024-01-04T12:15:00+00:00,2024-01-04T14:03:00+00:00,IAH,LAX,13


In [35]:
#Merge data
weather_delay_data = pd.merge(pressure_data, weather_data, on="airport")

# fix order of columns so that it is same as in training
weather_delay_data=weather_delay_data.reindex(sorted(weather_delay_data.columns), axis=1)

#merge columns
flight_weather_data=pd.merge(flight_data_api, weather_delay_data, on="airport")



In [36]:
#drop objects
flight_weather_data.drop(columns=['airport', 'origin', 'dest'], inplace=True)

In [37]:
flight_weather_data.info()
flight_weather_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   MONTH                      11 non-null     int64  
 1   DAY_OF_MONTH               11 non-null     int64  
 2   DAY_OF_WEEK                11 non-null     int64  
 3   origin_airport_id          11 non-null     int64  
 4   ORIGIN_WAC                 11 non-null     int64  
 5   dest_airport_id            11 non-null     int64  
 6   DEST_WAC                   11 non-null     int64  
 7   CRS_DEP_TIME               11 non-null     object 
 8   CRS_ARR_TIME               11 non-null     object 
 9   DEP_DELAY_NEW              11 non-null     int64  
 10  HourlyDewPointTemperature  11 non-null     float64
 11  HourlyDryBulbTemperature   11 non-null     float64
 12  HourlyPrecipitation        11 non-null     int64  
 13  HourlyRelativeHumidity     11 non-null     float64
 

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,DEP_DELAY_NEW,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,1,4,3,2,82,12,33,2024-01-04T10:33:00+00:00,2024-01-04T16:28:00+00:00,19,-6.7,-1.1,0,0.657199,1010.0,10,350.0,0,6
1,1,4,3,15,93,11,33,2024-01-04T08:10:00+00:00,2024-01-04T16:57:00+00:00,32,6.7,6.7,0,1.000000,1020.0,3,240.0,0,3
2,1,4,3,11,33,8,22,2024-01-04T13:30:00+00:00,2024-01-04T16:10:00+00:00,14,8.3,17.8,0,0.537274,1018.0,10,20.0,0,10
3,1,4,3,5,33,2,82,2024-01-04T15:15:00+00:00,2024-01-04T17:51:00+00:00,14,17.8,23.9,0,0.686854,1015.0,10,360.0,0,6
4,1,4,3,7,74,10,91,2024-01-04T12:15:00+00:00,2024-01-04T14:03:00+00:00,13,3.9,17.2,0,0.411786,1018.0,10,110.0,22,16


In [38]:
# Convert columns to datetime
flight_weather_data['CRS_DEP_TIME'] = pd.to_datetime(flight_weather_data['CRS_DEP_TIME'])
flight_weather_data['CRS_ARR_TIME'] = pd.to_datetime(flight_weather_data['CRS_ARR_TIME'])

# Extract time in HHMM format without a colon
flight_weather_data['CRS_DEP_TIME'] = flight_weather_data['CRS_DEP_TIME'].dt.strftime('%H%M')
flight_weather_data['CRS_ARR_TIME'] = flight_weather_data['CRS_ARR_TIME'].dt.strftime('%H%M')

In [39]:
columns_to_int64 = ['CRS_ARR_TIME', 'CRS_DEP_TIME', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']
columns_to_float64 = ['HourlyPrecipitation', 'HourlyVisibility', 'HourlyWindGustSpeed', 'HourlyWindSpeed', 'DEP_DELAY_NEW']

for column in columns_to_int64:
    # Convert to int64
    flight_weather_data[column] = flight_weather_data[column].astype('int64')
for column in columns_to_float64:
    # Convert to int64
    flight_weather_data[column] = flight_weather_data[column].astype('float64')

In [40]:
flight_weather_data.info()
flight_weather_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   MONTH                      11 non-null     int64  
 1   DAY_OF_MONTH               11 non-null     int64  
 2   DAY_OF_WEEK                11 non-null     int64  
 3   origin_airport_id          11 non-null     int64  
 4   ORIGIN_WAC                 11 non-null     int64  
 5   dest_airport_id            11 non-null     int64  
 6   DEST_WAC                   11 non-null     int64  
 7   CRS_DEP_TIME               11 non-null     int64  
 8   CRS_ARR_TIME               11 non-null     int64  
 9   DEP_DELAY_NEW              11 non-null     float64
 10  HourlyDewPointTemperature  11 non-null     float64
 11  HourlyDryBulbTemperature   11 non-null     float64
 12  HourlyPrecipitation        11 non-null     float64
 13  HourlyRelativeHumidity     11 non-null     float64
 

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,DEP_DELAY_NEW,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,1,4,3,2,82,12,33,1033,1628,19.0,-6.7,-1.1,0.0,0.657199,1010.0,10.0,350.0,0.0,6.0
1,1,4,3,15,93,11,33,810,1657,32.0,6.7,6.7,0.0,1.000000,1020.0,3.0,240.0,0.0,3.0
2,1,4,3,11,33,8,22,1330,1610,14.0,8.3,17.8,0.0,0.537274,1018.0,10.0,20.0,0.0,10.0
3,1,4,3,5,33,2,82,1515,1751,14.0,17.8,23.9,0.0,0.686854,1015.0,10.0,360.0,0.0,6.0
4,1,4,3,7,74,10,91,1215,1403,13.0,3.9,17.2,0.0,0.411786,1018.0,10.0,110.0,22.0,16.0


In [41]:
target_variable = flight_weather_data.pop('DEP_DELAY_NEW')

In [42]:
flight_weather_data.info()
flight_weather_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   MONTH                      11 non-null     int64  
 1   DAY_OF_MONTH               11 non-null     int64  
 2   DAY_OF_WEEK                11 non-null     int64  
 3   origin_airport_id          11 non-null     int64  
 4   ORIGIN_WAC                 11 non-null     int64  
 5   dest_airport_id            11 non-null     int64  
 6   DEST_WAC                   11 non-null     int64  
 7   CRS_DEP_TIME               11 non-null     int64  
 8   CRS_ARR_TIME               11 non-null     int64  
 9   HourlyDewPointTemperature  11 non-null     float64
 10  HourlyDryBulbTemperature   11 non-null     float64
 11  HourlyPrecipitation        11 non-null     float64
 12  HourlyRelativeHumidity     11 non-null     float64
 13  HourlyStationPressure      11 non-null     float64
 

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,origin_airport_id,ORIGIN_WAC,dest_airport_id,DEST_WAC,CRS_DEP_TIME,CRS_ARR_TIME,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
0,1,4,3,2,82,12,33,1033,1628,-6.7,-1.1,0.0,0.657199,1010.0,10.0,350.0,0.0,6.0
1,1,4,3,15,93,11,33,810,1657,6.7,6.7,0.0,1.000000,1020.0,3.0,240.0,0.0,3.0
2,1,4,3,11,33,8,22,1330,1610,8.3,17.8,0.0,0.537274,1018.0,10.0,20.0,0.0,10.0
3,1,4,3,5,33,2,82,1515,1751,17.8,23.9,0.0,0.686854,1015.0,10.0,360.0,0.0,6.0
4,1,4,3,7,74,10,91,1215,1403,3.9,17.2,0.0,0.411786,1018.0,10.0,110.0,22.0,16.0


In [43]:

target_variable.info()
target_variable.head()

<class 'pandas.core.series.Series'>
RangeIndex: 11 entries, 0 to 10
Series name: DEP_DELAY_NEW
Non-Null Count  Dtype  
--------------  -----  
11 non-null     float64
dtypes: float64(1)
memory usage: 216.0 bytes


0    19.0
1    32.0
2    14.0
3    14.0
4    13.0
Name: DEP_DELAY_NEW, dtype: float64

In [45]:
model.eval()

# Perform inference
with torch.no_grad():
    print(len(target_variable))
    for i in range(len(target_variable)):
        inference_data = torch.tensor(flight_weather_data.iloc[i].values, dtype=torch.float32)
        inference_data = scaler.transform(inference_data.reshape(1, -1))
        inference_data = torch.tensor(inference_data, dtype=torch.float32)
        #print(inference_data)
        y_pred=model(inference_data)
        try:
            print(f"Predicted departure delay in minutes: {int(y_pred.item())}. Real departure delay from flight API: {int(target_variable.iloc[i])}")
        except:
            print(f"Predicted departure delay in minutes: NaN. Real departure delay from flight API: {int(target_variable.iloc[i])}")

11
Predicted departure delay in minutes: 11. Real departure delay from flight API: 19
Predicted departure delay in minutes: 9. Real departure delay from flight API: 32
Predicted departure delay in minutes: 17. Real departure delay from flight API: 14
Predicted departure delay in minutes: 23. Real departure delay from flight API: 14
Predicted departure delay in minutes: 7. Real departure delay from flight API: 13
Predicted departure delay in minutes: 5. Real departure delay from flight API: 7
Predicted departure delay in minutes: 27. Real departure delay from flight API: 19
Predicted departure delay in minutes: 22. Real departure delay from flight API: 40
Predicted departure delay in minutes: 9. Real departure delay from flight API: 6
Predicted departure delay in minutes: NaN. Real departure delay from flight API: 39
Predicted departure delay in minutes: 8. Real departure delay from flight API: 10


If using year check manually how model is affected by a different year

In [25]:
flight_weather_data.loc[:, 'YEAR'] = 2022

In [26]:
model.eval()

# Perform inference
with torch.no_grad():
    print(len(target_variable))
    for i in range(len(target_variable)):
        inference_data = torch.tensor(flight_weather_data.iloc[i].values, dtype=torch.float32)
        inference_data = scaler.transform(inference_data.reshape(1, -1))
        inference_data = torch.tensor(inference_data, dtype=torch.float32)
        #print(inference_data)
        y_pred=model(inference_data)
        print("Predicted departure delay in minutes: " +str(y_pred.item())+". Real departure delay from flight API: "+str(target_variable.iloc[i]))

8
Predicted departure delay in minutes: 10.957042694091797. Real departure delay from flight API: 18.0
Predicted departure delay in minutes: 9.636273384094238. Real departure delay from flight API: 14.0
Predicted departure delay in minutes: 7.687645435333252. Real departure delay from flight API: 32.0
Predicted departure delay in minutes: 11.651840209960938. Real departure delay from flight API: 19.0
Predicted departure delay in minutes: 9.949142456054688. Real departure delay from flight API: 19.0
Predicted departure delay in minutes: 18.64274787902832. Real departure delay from flight API: 15.0
Predicted departure delay in minutes: 20.774188995361328. Real departure delay from flight API: 15.0
Predicted departure delay in minutes: nan. Real departure delay from flight API: 9.0
